<a href="https://colab.research.google.com/github/hanarayan/EPAM_PRACTICE/blob/main/practice_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
from pyspark import SparkContext
import csv,json , re
from io import StringIO
# Check if a SparkContext already exists
try:
    sc = SparkContext.getOrCreate()
    print("Using existing SparkContext")
except ValueError:
    # If not, create a new one
    sc = SparkContext("local", "Linkdin Example")
    print("Created a new SparkContext")


Using existing SparkContext


In [3]:
rdd  = sc.textFile("/content/LinkedIn people profiles datasets.csv")
rdd_raw = rdd.map(lambda line: line.replace(',"', '\t"'))
print (rdd_raw.take(2))

['"timestamp"\t"id"\t"name"\t"city"\t"country_code"\t"region"\t"current_company:company_id"\t"current_company:name"\t"position"\t"following"\t"about"\t"posts"\t"groups"\t"current_company"\t"experience"\t"url"\t"people_also_viewed"\t"educations_details"\t"education"\t"avatar"\t"languages"\t"certifications"\t"recommendations"\t"recommendations_count"\t"volunteer_experience"\t"сourses"', '"2023-01-10"\t"catherinemcilkenny"\t"Catherine Fitzpatrick (McIlkenny), B.A"\t"Canada"\t"CA"\t"NA"\t"null"\t""\t"Snr Business Analyst at Emploi et Développement social Canada (EDSC) / Employment and Social Development Canada (ESDC)"\t"null"\t"null"\t"[{""attribution"":""Liked by Catherine Fitzpatrick (McIlkenny), B.A""\t""img"":""https://media.licdn.com/dms/image/C5622AQGesy-cKVfCaA/feedshare-shrink_2048_1536/0/1668547197397?e=2147483647&v=beta&t=hK5L_Ky-Ox4m90KT5u-roJB4C6BtWmD5UnOxYRvMucI""\t""link"":""https://www.linkedin.com/posts/esdcedsc_esw2022-activity-6998394190682341376-nMQh""\t""title"":""It’s 

In [4]:
header = rdd_raw.first()
rdd_no_header = rdd_raw.filter(lambda line: line != header)
print(f"Total rows (Including header): {rdd_raw.count()}")
print(f"Total rows (excluding header): {rdd_no_header.count()}")

Total rows (Including header): 1001
Total rows (excluding header): 1000


In [5]:


header_columns = header.split("\t")
header_columns = [col.strip('""').strip().lower() for col in header_columns]

print("Available Columns:")
for idx, column in enumerate(header_columns):
    print(f"Index: {idx}, Column: {column}")



Available Columns:
Index: 0, Column: timestamp
Index: 1, Column: id
Index: 2, Column: name
Index: 3, Column: city
Index: 4, Column: country_code
Index: 5, Column: region
Index: 6, Column: current_company:company_id
Index: 7, Column: current_company:name
Index: 8, Column: position
Index: 9, Column: following
Index: 10, Column: about
Index: 11, Column: posts
Index: 12, Column: groups
Index: 13, Column: current_company
Index: 14, Column: experience
Index: 15, Column: url
Index: 16, Column: people_also_viewed
Index: 17, Column: educations_details
Index: 18, Column: education
Index: 19, Column: avatar
Index: 20, Column: languages
Index: 21, Column: certifications
Index: 22, Column: recommendations
Index: 23, Column: recommendations_count
Index: 24, Column: volunteer_experience
Index: 25, Column: сourses


In [6]:
country_code_index = header_columns.index("country_code")
name_index = header_columns.index("name")
region_index = header_columns.index("region")
company_name_index = header_columns.index("current_company:name")
following_index = header_columns.index("following")
post_index = header_columns.index("posts")
education_index = header_columns.index("education")
certification_index = header_columns.index("certifications")
recommendations_index = header_columns.index("recommendations_count")

In [7]:
#rdd_split = rdd_no_header.map(lambda line: line.split(","))

def parse_csv_line(lines):
    reader = csv.reader(lines, delimiter='\t')
    return [row for row in reader]

rdd_split = rdd_raw.filter(lambda row: row != header).mapPartitions(parse_csv_line)
print (rdd_split.take(1))

[['2023-01-10', 'catherinemcilkenny', 'Catherine Fitzpatrick (McIlkenny), B.A', 'Canada', 'CA', 'NA', 'null', '', 'Snr Business Analyst at Emploi et Développement social Canada (EDSC) / Employment and Social Development Canada (ESDC)', 'null', 'null', '[{"attribution":"Liked by Catherine Fitzpatrick (McIlkenny), B.A"\t"img":"https://media.licdn.com/dms/image/C5622AQGesy-cKVfCaA/feedshare-shrink_2048_1536/0/1668547197397?e=2147483647&v=beta&t=hK5L_Ky-Ox4m90KT5u-roJB4C6BtWmD5UnOxYRvMucI"\t"link":"https://www.linkedin.com/posts/esdcedsc_esw2022-activity-6998394190682341376-nMQh"\t"title":"It’s never too late to open an RESP. Find out how a little can go a long way for a child’s education and how saving can set them up for success!…"},{"attribution":"Liked by Catherine Fitzpatrick (McIlkenny), B.A"\t"img":"https://media.licdn.com/dms/image/C4E22AQHMWUhPL9AG5A/feedshare-shrink_800/0/1667668831230?e=2147483647&v=beta&t=LdI-tVu3yyWKpIm6MOsjYY0rsLMsLMjglOXvgjxPdyw"\t"link":"https://www.linkedi

In [8]:

def clean_column(code):
    cleaned_code = str(code).strip().strip('"')
    if not cleaned_code or cleaned_code.lower() in ["null", "no data","--"]:
        return "Not Available"
    return cleaned_code

In [9]:
def clean_digit(digit_value):
    try:
        return int(digit_value) if digit_value.isdigit() else 0
    except ValueError:
        return 0

In [10]:
print(country_code_index)
country_codes_rdd = rdd_split.map(lambda row: clean_column(row[country_code_index]))
distinct_countries = country_codes_rdd.distinct().sortBy(lambda x: x.lower()).collect()

print("Distinct Country Codes:")
for i,country in enumerate(distinct_countries, start=1):
    print(f"{i}. {country}")

4
Distinct Country Codes:
1. AE
2. AM
3. AR
4. AT
5. AU
6. BA
7. BE
8. BG
9. BH
10. BN
11. BR
12. BS
13. CA
14. CD
15. CH
16. CI
17. CL
18. CN
19. CO
20. CR
21. CZ
22. DE
23. DK
24. DZ
25. EG
26. ES
27. FI
28. FR
29. GB
30. GE
31. GH
32. GM
33. GR
34. HK
35. ID
36. IE
37. IL
38. IN
39. IQ
40. IR
41. IT
42. JP
43. KR
44. KW
45. LK
46. LV
47. MA
48. MK
49. MO
50. MX
51. MY
52. NG
53. NL
54. NO
55. Not Available
56. NZ
57. OM
58. PE
59. PH
60. PK
61. PL
62. PT
63. QA
64. RO
65. RS
66. RU
67. SA
68. SD
69. SE
70. SG
71. SK
72. SN
73. SY
74. TH
75. TR
76. TW
77. UK
78. US
79. UY
80. VE
81. VN
82. ZA
83. ZM


In [11]:
specific_country_code = "CA"
filtered_profiles_rdd = rdd_split.filter(lambda row: clean_column(row[country_code_index]) == specific_country_code)

# Extract names of profiles belonging to the specified country
profile_names_rdd = filtered_profiles_rdd.map(lambda row: clean_column(row[name_index]))

# Sort and collect unique profile names
sorted_profiles = profile_names_rdd.distinct().sortBy(lambda x: x.lower()).collect()

print(f"Profiles from {specific_country_code}:")
for i, profile in enumerate(sorted_profiles, start=1):
    print(f"{i}. {profile}")


Profiles from CA:
1. Angeline Lakusta
2. Arezo Hamid
3. Bernie Soto
4. Catherine Fitzpatrick (McIlkenny), B.A
5. Cesar Figueiredo
6. Cory MacDonald
7. Damoon Tahmasbi
8. Daniela Bruno
9. David-L. Nadeau
10. Dhwani Joshi
11. Dr. Mohamed Ali Kshad
12. Gerard Ludovic Wan
13. Harmeet Singh
14. Jean Charbonneau
15. Jim Battis
16. Kadari Mwene Kabyana
17. Kimberly Pleasance
18. Kirk Davies
19. Lindsey Boland
20. Lynn Millthorpe
21. Matthew Pritchard
22. Ned Flanders
23. Parminder Singh
24. Prikankshit Mor
25. Ryan MacLennan
26. Sankarshan Purkayastha
27. Sean C. Adelberg
28. Silver-Lee Kelly
29. Sirabi Joukakelian Zoboyan, PPCC
30. Steve Latimer
31. Todd Nelson
32. Tuan Pham
33. Vitaly Nhuien
34. Yurii Chetvertukha


In [12]:
regions_rdd  = rdd_split.map(lambda row: (clean_column(row[region_index]), 1))
region_counts = regions_rdd.reduceByKey(lambda a, b: a + b)
region_counts_result = region_counts.sortBy(lambda x: x).collect()

print("Region:" )
for region, count in region_counts_result:
    print(f"{region}, Count: {count}")


Region:
AF, Count: 26
AS, Count: 144
EU, Count: 209
NA, Count: 360
Not Available, Count: 184
OC, Count: 41
SA, Count: 36


In [13]:
company_names_rdd = rdd_split.map(lambda row: clean_column(row[company_name_index]))
distinct_company_names_rdd = company_names_rdd.distinct().sortBy(lambda x: x.lower()).collect()

print("Distinct company names:")
for i , company in enumerate(distinct_company_names_rdd,start=1):
     print(f"{i}. {company}")

Distinct company names:
1. 12th ave Thrift
2. 3BlindMiceUSA/Costco/Graber
3. 3M
4. A Local Hospital
5. A-Plan Insurance
6. A-reco Stockholm AB
7. A.S.C. ELEKTRİK MOTORLARI A.Ş.
8. ABC Therapy
9. ABN AMRO Bank N.V.
10. Absa Group
11. Abu Dhabi Commercial Bank
12. Abu Qir Petrolum
13. Academy of Holy Angels
14. Access Garage Doors
15. Accuspec Electronics, LLC
16. Aerotech Golf .LLC
17. African Reinsurance Brokers
18. African Spear
19. Afterburner
20. AGL Australia
21. Ailsa Sutcliffe Training Ltd.
22. Al Neem General Maintenance
23. Alcaldia de Cantagallo
24. ALDI Stores Australia
25. Alfardan Jewellery
26. Align Technology
27. All Personal, Inc
28. Allergan
29. Allianz Partners
30. Allied Dermatology
31. Alnylam Pharmaceuticals
32. Alshaya Group
33. Alten Sweden
34. Altour
35. Amarillo College
36. Amazon
37. Amdok
38. America dental
39. American Century Investments
40. American Heart Association
41. AMMSCPA
42. Analog Devices
43. ANAND AND ANAND
44. Animal Eye Guys
45. ANZ
46. Applied 

In [14]:
people_rdd = rdd_split.map(lambda row: (
    clean_digit(clean_column(row[following_index])),
    clean_column(row[name_index])
))


sorted_people_rdd = people_rdd.sortBy(lambda x: x[0], ascending=False)


top_10_followed = sorted_people_rdd.take(10)


print("Top 10 Most-Followed People:")
for i,(following, name) in enumerate(top_10_followed,start =1):
    print(f"{i}. {name}: {following} followers")


Top 10 Most-Followed People:
1. Eloise Pamela Pusch: 617 followers
2. Hashan Rathnayaka: 515 followers
3. Giovanna Panarella: 500 followers
4. Jessica McCray: 500 followers
5. Andres Jaramillo Moyano: 500 followers
6. Jörgen Domby: 500 followers
7. Claire Sykes: 500 followers
8. Julia Garcia Antenucci: 500 followers
9. Prateek Chitpur: 500 followers
10. Christian Zerial: 500 followers


In [15]:
def clean_string(raw_str):
    """Cleans raw JSON string by fixing formatting issues."""
    cleaned_str = raw_str.strip()  # Remove leading/trailing spaces
    cleaned_str = cleaned_str.replace("\t", ",")  # Fix delimiters
    cleaned_str = cleaned_str.replace("['", "[")  # Fix opening bracket
    cleaned_str = cleaned_str.replace("']", "]")  # Fix closing bracket
    cleaned_str = cleaned_str.replace('""', 'null')  # Handle empty values correctly
    return cleaned_str

def safe_json_loads(raw_str):
    """Safely loads JSON string, handling decoding errors."""
    try:
        return json.loads(raw_str)
    except json.JSONDecodeError as e:
        print(f"JSON Parse Error: {raw_str} | Error: {e}")  # Debugging output
        return None  # Return None for invalid JSON rows

In [16]:
def extract_profile_likes(row):
    """Extracts profile name, falling back to 'name' column if attribution is missing."""
    post_list = safe_json_loads(clean_string(row[post_index]))

    if post_list:
        extracted_data = []
        for post in post_list:
            profile_name = post.get("attribution", row[name_index]).replace("Liked by ", "") if post.get("attribution") else row[name_index]
            extracted_data.append((profile_name, 1))
        return extracted_data
    return []

profile_likes_rdd = rdd_split.flatMap(extract_profile_likes)

likes_count_rdd = profile_likes_rdd.reduceByKey(lambda a, b: a + b)

all_profiles_rdd = rdd_split.map(lambda row: row[name_index]).distinct()

likes_count_dict = dict(likes_count_rdd.collect())

final_sorted_likes_count = sorted([(profile, likes_count_dict.get(profile, 0)) for profile in all_profiles_rdd.collect()], key=lambda x: x[0])

print("Total Posts Liked Per Profile (Sorted by Name, Including 0 Likes):")
for profile, count in final_sorted_likes_count:
    print(f"{profile}: {count}")

Total Posts Liked Per Profile (Sorted by Name, Including 0 Likes):
(Dr) Rogério Rondini: 0
A. Gökhan Çimen: 0
AJ Godbolt: 0
AKCE Cont. Est.: 3
ANABEL PANGALANGAN: 0
ANGEL SOLIS QUIJADA: 0
Aaron Rodzinak: 3
Aaron Sandomire: 0
Aaron Zola: 0
Abigail Acierto: 3
Adam Buckley M.A.: 3
Adam Goto: 3
Adam Tarbox: 0
Adarsh Raj: 0
Adelia Irma Pamasih: 3
Adil Grini: 0
Adrian Epps: 0
Adrian Maniatis: 0
Ahida Suljkanović-Mahmutović: 0
Ahmad Alshugairi: 0
Ahmad Fauzi Harun: 0
Ailsa Sutcfiffe: 0
Aimeé Eckl: 3
Ajay Malik: 0
Ajaz Khan: 0
Alaa El-said: 0
Alan Brockington: 0
Alan Queiroz: 0
Albert Rubinov: 0
Aldin John Coquilla: 0
Aleah Vaske: 2
Alef Almeida Chaves: 3
Alessandra Panseri: 0
Alex Marin: 0
Alexander Stratz: 2
Alexander Zhura: 0
Alexandra Franks: 0
Alexandra GAZIGLIA: 0
Alexandra Gardner: 3
Alexandra Goldfarb: 0
Alexandre Campos: 3
Alexandru Neacsu: 0
Alfredo Rueda: 3
Ali Kia: 0
Alisa Joseph: 3
Alisa Poe: 3
Allan Clayton: 0
Allison Becker: 0
Alma Rose Daaca: 3
Amandine Ruchon: 0
Ameen Zakariya

In [17]:
# Apply cleaning and JSON transformation
education_rdd = rdd_split.map(lambda row: safe_json_loads(clean_string(row[education_index]))).filter(lambda x: x is not None)

# Debugging output to verify JSON parsing
print("Cleaned JSON Output:", education_rdd.take(10))

def extract_education_profiles(row):
    """Extracts education institutions from 'title' tag within JSON, linked to profiles."""
    education_list = safe_json_loads(clean_string(row[education_index]))  # Convert 'education' column to JSON

    if education_list:  # Ensure JSON conversion is valid
        extracted_data = []
        for education in education_list:
            institution = education.get("title", "Unknown Institution")  # Get institution name
            extracted_data.append((institution, 1))  # Count occurrences
        return extracted_data
    return []  # Return empty list if no valid education data

# Create RDD mapping education institutions to profile counts
education_count_rdd = rdd_split.flatMap(extract_education_profiles)

# **Aggregate counts for each institution**
institution_counts_rdd = education_count_rdd.reduceByKey(lambda a, b: a + b)

# **Extract all unique education institutions to ensure missing entries are shown with `0` counts**
all_institutions_rdd = education_rdd.flatMap(lambda edu_list: [edu.get("title", "Unknown Institution") for edu in edu_list]).distinct()

# Convert counts to a dictionary for lookup
institution_count_dict = dict(institution_counts_rdd.collect())

# **Ensure missing institutions appear with a count of `0`**
final_sorted_institutions = sorted([(institution, institution_count_dict.get(institution, 0)) for institution in all_institutions_rdd.collect()], key=lambda x: x[0].lower())

# **Print final sorted results**
print("Total Profiles by Education Institution (Sorted Alphabetically):")
for institution, count in final_sorted_institutions:
    print(f"{institution}: {count}")

Cleaned JSON Output: [[{'degree': 'Bachelor of Arts (B.A.) Honours', 'end_year': '2011', 'field': 'English Language and Literature', 'meta': '2009 - 2011', 'start_year': '2009', 'title': "Queen's University Belfast", 'url': "https://www.linkedin.com/school/queen's-university-belfast/"}, {'degree': 'Business Analysis Essentials Certification', 'end_year': '2021', 'field': 'Business Analysis', 'meta': '2019 - 2021 The Business Analysis Essentials program delivers practical, hands-on, experience-based training in the core knowledge and skills of business analysis. Courses: Fundamentals of Business AnalysisAccelerated Requirements, Elicitation and Analysis Facilitation Techniques for Business AnalystsModelling Business Processes and Workflows Use Case Modelling Validating and Testing Requirements Business Analysis in an Agile Environment', 'start_year': '2019', 'title': 'Algonquin College of Applied Arts and Technology', 'url': 'https://www.linkedin.com/school/algonquincollege/'}, {'degree

In [18]:
# Apply cleaning and JSON transformation
certification_rdd = rdd_split.map(lambda row: safe_json_loads(clean_string(row[certification_index]))).filter(lambda x: x is not None)

# Debugging output to verify JSON parsing
print("Cleaned JSON Output:", certification_rdd.take(10))

def extract_certification_profiles(row):
    """Extracts certification names from 'title' tag within JSON, linked to profiles."""
    certification_list = safe_json_loads(clean_string(row[certification_index]))  # Convert 'certification' column to JSON

    if certification_list:  # Ensure JSON conversion is valid
        extracted_data = []
        for certification in certification_list:
            cert_name = certification.get("title", "Unknown certifications")  # Get certification name
            extracted_data.append((cert_name, 1))  # Count occurrences
        return extracted_data
    return []  # Return empty list if no valid certification data

# Create RDD mapping certifications to profile counts
certification_count_rdd = rdd_split.flatMap(extract_certification_profiles)

# **Aggregate counts for each certification**
certification_counts_rdd = certification_count_rdd.reduceByKey(lambda a, b: a + b)

# **Extract all unique certifications to ensure missing entries are shown with `0` counts**
all_certifications_rdd = certification_rdd.flatMap(lambda cert_list: [cert.get("title", "Unknown certifications") for cert in cert_list]).distinct()

# Convert counts to a dictionary for lookup
certification_count_dict = dict(certification_counts_rdd.collect())

# **Ensure missing certifications appear with a count of `0`**
final_sorted_certifications = sorted([(cert_name, certification_count_dict.get(cert_name, 0)) for cert_name in all_certifications_rdd.collect()], key=lambda x: x[0].lower())

# **Print final sorted results**
print("Total Profiles by certifications (Sorted Alphabetically):")
for cert_name, count in final_sorted_certifications:
    print(f"{cert_name}: {count}")

Cleaned JSON Output: [[{'meta': 'Issued Jun 2013', 'subtitle': 'Van der Hilst Communicatie', 'title': 'Senior Communicatieprofessional C (BNP4)'}], [{'meta': 'Issued Jan 2022 See credential', 'subtitle': 'IBM', 'title': 'IBM Power Systems Enterprise Linux Technical V2'}, {'meta': 'Issued Jan 2022 See credential', 'subtitle': 'IBM', 'title': 'IBM Power Systems Infrastructure as a Service Technical V2'}, {'meta': 'Issued Jan 2022 See credential', 'subtitle': 'IBM', 'title': 'IBM Power Systems Solution Foundation'}, {'meta': 'Issued Jan 2021 See credential', 'subtitle': 'IBM', 'title': 'IBM Power Systems Infrastructure Sales Foundation V2'}, {'meta': 'Issued Jan 2021 See credential', 'subtitle': 'IBM', 'title': 'IBM Power Systems Infrastructure Sales Foundation V2'}, {'meta': 'Issued Dec 2020 See credential', 'subtitle': 'IBM', 'title': 'Cloud Pak for Data Sales Foundations'}, {'meta': 'Issued Dec 2020 See credential', 'subtitle': 'IBM', 'title': 'IBM Journey to Cloud Sales Essentials'}, 

In [19]:
# Function to parse and handle null values
def extract_recommendations(row):
    """Extracts profile names and their corresponding recommendation counts."""
    name = row[name_index]  # Extract profile name
    recommendations = row[recommendations_index]  # Extract recommendations count

    # Convert recommendations to integer safely, treating null as 0
    recommendations = int(recommendations) if recommendations and recommendations.isdigit() else 0

    return (name, recommendations)

# Create RDD with extracted recommendation counts
recommendations_rdd = rdd_split.map(extract_recommendations)

# **Aggregate total recommendations per profile**
total_recommendations_rdd = recommendations_rdd.reduceByKey(lambda a, b: a + b)

# **Sort results alphabetically by profile name**
sorted_recommendations = total_recommendations_rdd.sortByKey()

# **Print final sorted results**
print("Total Recommendations per Profile (Sorted Alphabetically):")
for name, count in sorted_recommendations.collect():
    print(f"{name}: {count}")

Total Recommendations per Profile (Sorted Alphabetically):
(Dr) Rogério Rondini: 9
A. Gökhan Çimen: 0
AJ Godbolt: 3
AKCE Cont. Est.: 0
ANABEL PANGALANGAN: 0
ANGEL SOLIS QUIJADA: 0
Aaron Rodzinak: 0
Aaron Sandomire: 0
Aaron Zola: 0
Abigail Acierto: 0
Adam Buckley M.A.: 0
Adam Goto: 0
Adam Tarbox: 2
Adarsh Raj: 9
Adelia Irma Pamasih: 0
Adil Grini: 0
Adrian Epps: 0
Adrian Maniatis: 0
Ahida Suljkanović-Mahmutović: 0
Ahmad Alshugairi: 0
Ahmad Fauzi Harun: 0
Ailsa Sutcfiffe: 0
Aimeé Eckl: 0
Ajay Malik: 18
Ajaz Khan: 0
Alaa El-said: 9
Alan Brockington: 0
Alan Queiroz: 1
Albert Rubinov: 0
Aldin John Coquilla: 0
Aleah Vaske: 7
Alef Almeida Chaves: 0
Alessandra Panseri: 0
Alex Marin: 1
Alexander Stratz: 0
Alexander Zhura: 3
Alexandra Franks: 0
Alexandra GAZIGLIA: 0
Alexandra Gardner: 0
Alexandra Goldfarb: 0
Alexandre Campos: 0
Alexandru Neacsu: 0
Alfredo Rueda: 0
Ali Kia: 2
Alisa Joseph: 0
Alisa Poe: 0
Allan Clayton: 0
Allison Becker: 0
Alma Rose Daaca: 0
Amandine Ruchon: 0
Ameen Zakariya: 1
Ami